In [19]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

import time
import json
from datetime import datetime

import requests
from bs4 import BeautifulSoup

In [20]:
caps = DesiredCapabilities.CHROME
caps['goog:loggingPrefs'] = {'performance': 'ALL'}

options = Options()

options.add_experimental_option("detach", True)

options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36")

driver = webdriver.Chrome(options=options)

In [21]:
url = 'https://www.google.com/finance/?hl=ja'
driver.get(url)

time.sleep(2)

logs = driver.get_log('performance')

In [22]:
stock_code_input = driver.find_element(By.XPATH, "/html/body/c-wiz[2]/div/div[3]/div[3]/div/div/div/div[1]/input[2]")
stock_code_input.send_keys("NASDAQ")
stock_code_input.send_keys(Keys.RETURN)

In [24]:
driver.find_element(By.ID, '6monthTab').click()

In [25]:
batch_request_url = None
headers = {}
post_data = None

stock_code = 'NASDAQ'

for entry in logs:
    log = json.loads(entry['message'])['message']
    try:
        if 'Network.requestWillBeSent' in log['method']:
            request_url = log['params']['request']['url']
            if 'https://www.google.com/finance/_/GoogleFinanceUi/data/batchexecute?rpcids=' in request_url and stock_code in request_url:
                batch_request_url = request_url
                headers = log['params']['request']['headers']
                if 'postData' in log['params']['request']:
                    post_data = log['params']['request']['postData']
                break
    except KeyError:
        continue

# driver.quit()
print(batch_request_url)

None


In [ ]:
if batch_request_url:
    print(batch_request_url)
    if post_data:
        response = requests.post(batch_request_url, headers=headers, data=post_data)
    else:
        response = requests.get(batch_request_url, headers=headers)

    print("Status Code:", response.status_code)
    print("Response Text:", response.text)
    try:
        data = response.json()
        print(data)
    except json.JSONDecodeError:
        print("レスポンスのJSONデコードに失敗しました")
else:
    print("バッチリクエストURLが見つかりませんでした")

バッチリクエストURLが見つかりませんでした


In [ ]:
def remove_before_bracket_and_after_newline(s):
    # '['前の文字を削除
    bracket_index = s.find('[')
    if bracket_index != -1:
        s = s[bracket_index:]

    # 改行以降の文字を削除
    newline_index = s.find('\n')
    if newline_index != -1:
        s = s[:newline_index]

    return s

def parse_array_string(array_string):
    def parse_helper(s):
        # 空白文字を除去
        s = s.strip()
        if s.startswith('[') and s.endswith(']'):
            s = s[1:-1]
        else:
            return s.strip()  # Base case: return the stripped string itself

        result = []
        nested_level = 0
        start_index = 0

        for i, char in enumerate(s):
            if char == '[':
                if nested_level == 0:
                    start_index = i
                nested_level += 1
            elif char == ']':
                nested_level -= 1
                if nested_level == 0:
                    result.append(parse_helper(s[start_index:i+1]))
            elif char == ',' and nested_level == 0:
                result.append(parse_helper(s[start_index:i]))
                start_index = i + 1

        if start_index < len(s):
            result.append(parse_helper(s[start_index:]))

        return result

    return parse_helper(array_string)


res_text = remove_before_bracket_and_after_newline(response.text)

parsed_array = parse_array_string(res_text)

# print(res_text)
print(type(res_text))
print(type(parsed_array))
print(len(parsed_array[0][2][0][0][4][0][2]))
print(parsed_array[0][2][0][0][4][0][2][0])
print(parsed_array[0][2][0][0][4][0][2][0][0])
print(parsed_array[0][2][0][0][4][0][2][0][0][0])
print(parsed_array[0][2][0][0][4][0][2][0][0][1])
print(parsed_array[0][2][0][0][4][0][2][0][0][2])
print(parsed_array[0][2][0][0][4][0][2][0][2][0])

<class 'str'>
<class 'list'>
252
[['2024', '1', '23', '16', 'null', 'null', 'null', ['-14400'], ['-14400']], ['2024', '1', '23', '16', 'null', 'null', 'null', ['-14400'], ['-14400']], ['59.87', '0', '0', '2', '2', '2'], ['59.87', '0', '0', '2', '2', '2'], '29465445']
['2024', '1', '23', '16', 'null', 'null', 'null', ['-14400'], ['-14400']]
2024
1
23
59.87


In [ ]:
for data_array in parsed_array[0][2][0][0][4][0][2]:
    print(data_array[0][0], data_array[0][1], data_array[0][2], data_array[2][0])

2024 1 23 59.87
2024 1 23 59.87
2024 1 24 61.36
2024 1 24 61.36
2024 1 25 61.62
2024 1 25 61.62
2024 1 26 61.03
2024 1 26 61.03
2024 1 29 62.47
2024 1 29 62.47
2024 1 30 62.77
2024 1 30 62.77
2024 1 31 61.53
2024 1 31 61.53
2024 2 1 63.03
2024 2 1 63.03
2024 2 2 66.16
2024 2 2 66.16
2024 2 5 69.33
2024 2 5 69.33
2024 2 6 68.22
2024 2 6 68.22
2024 2 7 70.1
2024 2 7 70.1
2024 2 8 69.64
2024 2 8 69.64
2024 2 9 72.13
2024 2 9 72.13
2024 2 12 72.25
2024 2 12 72.25
2024 2 13 72.13
2024 2 13 72.13
2024 2 14 73.9
2024 2 14 73.9
2024 2 15 72.66
2024 2 15 72.66
2024 2 16 72.61
2024 2 16 72.61
2024 2 20 69.45
2024 2 20 69.45
2024 2 21 67.47
2024 2 21 67.47
2024 2 22 78.54
2024 2 22 78.54
2024 2 23 78.82
2024 2 23 78.82
2024 2 26 79.09
2024 2 26 79.09
2024 2 27 78.7
2024 2 27 78.7
2024 2 28 77.66
2024 2 28 77.66
2024 2 29 79.11
2024 2 29 79.11
2024 3 1 82.28
2024 3 1 82.28
2024 3 4 85.24
2024 3 4 85.24
2024 3 5 85.96
2024 3 5 85.96
2024 3 6 88.7
2024 3 6 88.7
2024 3 7 92.67
2024 3 7 92.67
2024 3 8